In [1]:
import basicLib.loadAndTest as orig
import basicLib.featureUpdate as featUp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import *
from pylab import *
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import re
from sklearn.linear_model import LogisticRegression
import basicLib.modelRun as runms
from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time
import clf2net
import clf2netTest
import clf2netTree
from sklearn import ensemble

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
path = '../input_clean/users_2014_in_sessions.csv'
usersTrain = orig.loadAndUpdateFeatures(path)
featureListTrain = orig.featureList(myList = ('affiliate_channel','affiliate_provider','first_affiliate_tracked',
                           'first_browser','first_device_type','language','signup_app','signup_method',
                           'signup_flow','date_account_created_dayofweek','date_first_active_dayofweek','age',
                           'validAge','gender'), usersCol=usersTrain.columns)
orig.initiateUsers(usersTrain)
Xtrain = orig.sortFeatures(usersTrain,featureListTrain.get())

path = '../input_clean/test_users.csv'
featureStruct = featUp.generateFeatureStruct()
del featureStruct[3]
featureStruct[3] = featUp.categoryFeature(categorical_features=('affiliate_channel',
                                                   'affiliate_provider',
                                                   'first_affiliate_tracked',
                                                   'first_browser',
                                                   'first_device_type',
                                                   'gender',
                                                   'language',
                                                   'signup_app',
                                                   'signup_method',
                                                   'signup_flow')   )
usersTest = orig.loadAndUpdateFeatures(path, featureStruct=featureStruct)
featureListTest = orig.featureList(myList = ('affiliate_channel','affiliate_provider','first_affiliate_tracked',
                           'first_browser','first_device_type','language','signup_app','signup_method',
                           'signup_flow','date_account_created_dayofweek','date_first_active_dayofweek','age',
                           'validAge','gender'), usersCol=usersTest.columns)
orig.initiateUsers(usersTest)
Xtest = orig.sortFeatures(usersTest,featureListTest.get())

In [ ]:
pd.concat([Xtrain,Xtest],ignore_index=True)

In [ ]:
X_byteDF = orig.category2binaryFeatures(pd.concat([d1,d2],ignore_index=True))

In [28]:
for i in range(len(Xtest.columns)):
    attrTest = getattr(Xtest,Xtest.columns[i])
    attrTrain = getattr(Xtrain,Xtrain.columns[i])
    for x in attrTest.unique():
        if x not in attrTrain.unique():
            print(Xtest.columns[i],':', x)

first_browser : IBrowse
first_browser : Nintendo Browser
first_browser : UC Browser
language : -unknown-
signup_method : weibo
signup_flow : 14
age : 0.396201240981


In [41]:
for x in X_byteDF.columns:
    print(x)

affiliate_channel_0
affiliate_channel_1
affiliate_channel_2
affiliate_channel_3
affiliate_channel_4
affiliate_channel_5
affiliate_channel_6
affiliate_provider_0
affiliate_provider_1
affiliate_provider_2
affiliate_provider_3
affiliate_provider_4
affiliate_provider_5
affiliate_provider_6
affiliate_provider_7
affiliate_provider_8
affiliate_provider_9
affiliate_provider_10
affiliate_provider_11
affiliate_provider_12
affiliate_provider_13
affiliate_provider_14
affiliate_provider_15
affiliate_provider_16
first_affiliate_tracked_0
first_affiliate_tracked_1
first_affiliate_tracked_2
first_affiliate_tracked_3
first_affiliate_tracked_4
first_affiliate_tracked_5
first_affiliate_tracked_6
first_affiliate_tracked_7
first_browser_0
first_browser_1
first_browser_2
first_browser_3
first_browser_4
first_browser_5
first_browser_6
first_browser_7
first_browser_8
first_browser_9
first_browser_10
first_browser_11
first_browser_12
first_browser_13
first_browser_14
first_browser_15
first_browser_16
first_bro

## new try

In [15]:
pathTrain = '../input_clean/users_train_norm.csv'
pathTest = '../input_clean/users_test_norm.csv'

usersTrain = pd.read_csv(pathTrain)
usersTest = pd.read_csv(pathTest)
featureStruct=featUp.generateFeatureStruct()
del featureStruct[3]
featureStruct[3] = featUp.categoryFeature(categorical_features=('affiliate_channel',
                                                   'affiliate_provider',
                                                   'first_affiliate_tracked',
                                                   'first_browser',
                                                   'first_device_type',
                                                   'gender',
                                                   'language',
                                                   'signup_app',
                                                   'signup_method',
                                                   'signup_flow')   )
yRes = usersTrain['country_destination']
usersTrain = usersTrain.drop('country_destination',axis=1)

users = pd.concat([usersTrain,usersTest],ignore_index=True)
for feature in featureStruct:
    feature.update(users)

featureList = orig.featureList(myList = ('affiliate_channel','affiliate_provider','first_affiliate_tracked',
                           'first_browser','first_device_type','language','signup_app','signup_method',
                           'signup_flow','date_account_created_dayofweek','date_first_active_dayofweek','age',
                           'validAge','gender'), usersCol=usersTrain.columns)

orig.initiateUsers(users)
Xtest = orig.sortFeatures(users,featureList.get())

In [3]:
Xlabel = Xtest
nonCategoryList=[]
for feature, dtype in Xlabel.dtypes.iteritems():
    if dtype.name != 'category':
        nonCategoryList.append(feature)
nonCategoryDF = Xlabel[nonCategoryList]
Xlabel = Xlabel.drop(nonCategoryList, 1)

labels = Xlabel.columns
myLabels = {} # holds the encode labels with value between 0 and n-1
n_values = [] # holds the length of each feature in bits ('gender' with 4 option will be 4 bits)
X = []
for label in Xlabel:
    labelAttr = getattr(Xlabel, label)
    myLabels[label] = preprocessing.LabelEncoder()
    myLabels[label].fit(labelAttr)
    if size(labelAttr.unique()) > 400:
        raise TypeError('The feature:', label, 'is too big to handle as binary')
    n_values.append(size(labelAttr.unique()))
    #transform: turn options into 0 to n-1,  reshape+transpose: align the coulumn so it be easier to add them to the table 
    X.append(myLabels[label].transform(labelAttr).reshape(1,len(labelAttr)).transpose())

#There is no category to change

temp = np.array([]).reshape(len(X[0]),0)
for i in range(len(X)):
    temp = concatenate((temp,X[i]),axis=1)
X = temp

enc = preprocessing.OneHotEncoder(n_values=n_values)
enc.fit(X)
X_byte = enc.transform(X).toarray()

X_byteDF = orig.makeDataFrame(X_byte,labels,n_values)
result = pd.concat([X_byteDF, nonCategoryDF], axis=1)

In [4]:
myDrop = {}
myList=('affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
        'first_browser', 'first_device_type', 'gender', 'language',
        'signup_app', 'signup_method', 'signup_flow')
for i in myList:
    attrTest = getattr(usersTest,i)
    attrTrain = getattr(usersTrain,i)
    myDrop[i] = []
    for x in attrTest.unique():
        if x not in attrTrain.unique():
            myDrop[i].append(x)
            print(i,':', x)

first_affiliate_tracked : nan
first_browser : Opera Mobile
first_browser : IBrowse
first_browser : Nintendo Browser
first_browser : CometBird
first_browser : UC Browser
language : -unknown-
signup_method : weibo
signup_flow : 14


In [5]:
for key in myDrop.keys():
    for testCat in myDrop[key]:
        print(key,',', testCat)
        if pd.isnull(testCat): continue
        myInd = myLabels[key].classes_.tolist().index(testCat)
        label = key + '_' + str(myInd)
        result = result.drop(label,axis=1)

first_affiliate_tracked , nan
first_browser , Opera Mobile
first_browser , IBrowse
first_browser , Nintendo Browser
first_browser , CometBird
first_browser , UC Browser
language , -unknown-
signup_flow , 14
signup_method , weibo


In [6]:
orig.normDf(result)

In [7]:
finalTrain = result[0:len(yRes)]
finalTest = result[len(yRes):]

In [8]:
len(finalTest)

62096

In [9]:
X_train, X_test, y_train, y_test = train_test_split(finalTrain, yRes)

### inner test

In [10]:
predictMethod = LogisticRegression()
predictMethod.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
y_pred = predictMethod.predict(X_test)
accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod.predict_proba(X_test)

In [12]:
accuracy_score

0.69442939200173404

In [13]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    myArr.append(predictMethod.classes_[order[:5]])

In [14]:
y_true = np.array(y_test)
mySum=0
for index in range(len(myArr)):
    for i in range(len(myArr[index])):
        if(myArr[index][i]==y_true[index]):
            mySum += 1/np.log2(i+2)
mySum/len(y_true)

0.84987223364960196

### send to airbnb

In [15]:
predictMethod = LogisticRegression()
predictMethod.fit(finalTrain,yRes)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
y_pred = predictMethod.predict(finalTest)
#accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod.predict_proba(finalTest)

In [17]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    for i in predictMethod.classes_[order[:5]]:
        myArr.append(i)

In [23]:
idCol = usersTest.id.repeat(5)
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol , 'country' : pd.Series(myArr)}

In [24]:
df = pd.DataFrame(d,columns=['id', 'country'])

In [25]:
df

,id,country
0,5uwns89zht,NDF
1,5uwns89zht,US
2,5uwns89zht,other
3,5uwns89zht,FR
4,5uwns89zht,ES
5,jtl0dijy2j,NDF
6,jtl0dijy2j,US
7,jtl0dijy2j,other
8,jtl0dijy2j,FR
9,jtl0dijy2j,CA


In [26]:
df.to_csv('../input_clean/submission_sessions_regression.csv',index=False)